In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,3211.39,-0.01,0.00,-0.01,-0.01,-0.00,-0.00,0.00
3,-0.01,856.94,-0.00,0.01,0.00,0.00,0.00,-0.00
4,0.00,-0.00,197.21,-0.00,-0.00,0.00,-0.00,0.00
5,-0.01,0.01,-0.00,43.88,0.00,0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,9.09,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,1.87,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.43,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00001,-0.00002,-0.00004,-0.00001,-0.00004,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00003,0.00000,-0.00003
4,0.00001,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00002
5,-0.00002,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00002,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00001,0.00003,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00004,0.00000,-0.00002,0.00002,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00002,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.027238434231904158

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.29124452e+06 8.99285154e+04 4.63236855e+03 2.20202736e+02
 8.87733049e+00 3.35581456e-01 1.42301265e-02 4.96775871e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999123,-0.040917,-0.008686,-0.001928,-0.000401,-0.000082,-0.000019,-0.000005
1,0.040512,0.998308,-0.040785,-0.008533,-0.001763,-0.000361,-0.000084,-0.000024
2,0.010217,0.039938,0.997996,-0.047083,-0.009202,-0.001878,-0.000438,-0.000120
3,0.002720,0.010197,0.046094,0.997341,-0.054373,-0.010595,-0.002448,-0.000681
4,0.000702,0.002622,0.011424,0.052932,0.995827,-0.071585,-0.015689,-0.004334
5,0.000188,0.000703,0.003056,0.013773,0.068340,0.989735,-0.120795,-0.031027
6,0.000061,0.000225,0.000982,0.004406,0.021295,0.106612,0.956041,-0.272298
7,0.000036,0.000135,0.000585,0.002631,0.012682,0.061787,0.266726,0.961703


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.000877156402963819,
 0.0016917771754521027,
 0.002004337670686196,
 0.002659125729508549,
 0.004173214407765968,
 0.010264672859782609,
 0.04395933501782967,
 0.038297484990094444]